# 1. The notebook for creating single vertebrae dataset out of full spine dataset

## 1.1 Description of the original labels

* Original markdown data from 3D Slicer contains a lot of information not necessary for ground truth creation and use. 

* Also it contains markdown for full spine, while what we need now is a set of markdowns of each vertebrae in a spine.

* Run this notebook on original (non-resized data)

Either we can firstly change the original markdown, in according to resized images (like, substract specific amount om millimeters from each point coordinates in markdown) and only after that proceed to this notebook and create data for yolo without any difficulties

In [ ]:
import os
import numpy as np
import json
from functionality import *
from operator import itemgetter
import PIL
from PIL import Image, ImageDraw, ImageColor, ImageFont
import pydicom


In [ ]:
# path_to_dataset = "C:\\Users\\gieko\\Dropbox\\NIITO_Vertebrae\\NIITO_Vertebrae_Dataset\\NIITO_Vertebrae_Dataset_Final_resized"
path_to_dataset = "C:\\Users\\gieko\\Dropbox\\NIITO_Vertebrae\\NIITO_Vertebrae_Dataset\\NIITO_Vertebrae_Dataset_Final_Test_resized"

path_to_images = os.path.join(path_to_dataset, "dicom")
path_to_labels = os.path.join(path_to_dataset, "labels")

# path_to_image_001 = os.path.join(path_to_images, "001")
# path_to_labels_001 = os.path.join(path_to_labels, *["001", "001_SD"])

In [ ]:
cases = os.listdir(path_to_labels)
# idexes = [1,2,5,6,7,8,9,10,18,20,21,31,40,45]
# cases[2]
# kek = [ cases[i] for i in idexes]
cases

['053_SD', '057_SD', '060_SD', '063_SD', '073_SD']

In [ ]:
# from alive_progress import alive_bar
palette_color = {
    "C": "#ff0000",
    "T": "#ff7128",
    "L": "#ffcc00",
    "S": "#92d050",
    "F": "#00b0f0"
}

cases = os.listdir(path_to_labels)
cases
# cases = ["001"]

show_exaple = True
data_coordinates = []

for case in cases:
    path_to_orig_image = os.path.join(path_to_images, case + ".dcm")
    point_names = ["CRV", "CAV", "CRD", "CAD"]
    # print(case, path_to_orig_image)
    spacing = None

    dicom = pydicom.dcmread(path_to_orig_image)
    input = Image.fromarray(dicom.pixel_array)
    input = input.convert('RGB')
    # input = get_PIL_image(dicom)
    # print(input)
    # plt.figure(figsize=(50, 30))
    if spacing == None:
        spacing = dicom.ImagerPixelSpacing
    page_width, page_height = input.size
    markdowns = read_all_markdowns(os.path.join(path_to_labels, case))
    # mask = Image.new('RGB', (width, height), 'black')
    palette = palette_color
    # draw = ImageDraw.Draw(input)
    for k, markdown in enumerate(markdowns):
        filename = case + ".png"
        id = k
        AuthorID, Overlapped = ('NA', 'TRUE')

        if "FH"  not in markdown['name']:
            category = 'Vertebrae' if "C2" not in markdown['name'] else 'C2'
            coordinates = [(float(i['position'][0]) / spacing[0], float(i['position'][1]) / spacing[1]) for i in markdown['controlPoints']]

            min_x, max_x = min(coordinates, key=itemgetter(0))[0], max(coordinates, key=itemgetter(0))[0]
            min_y, max_y = min(coordinates, key=itemgetter(1))[1], max(coordinates, key=itemgetter(1))[1]

        else:
            category = 'FermutHead'
            coordinates = [(float(i['position'][0]), float(i['position'][1])) for i in markdown['controlPoints']]
            c, r = define_circle(coordinates[0], coordinates[1], coordinates[2])
            c, r = (c[0] / spacing[0], c[1] / spacing[1]), r / spacing[0]

            min_x, max_x = c[0] - r, c[0] + r
            min_y, max_y = c[1] - r, c[1] + r
        
        category1 = markdown['name']
        width = max_x - min_x
        height = max_y - min_y
        x, y = int(min_x), int(min_y)
        w, h = int(width), int(height)

    #     # coordinates1 = [(min_x - 2, min_y - 2), (min_x - 2, max_y + 2), (max_x + 2, max_y + 2), (max_x + 2, min_y - 2)]
        coordinates2 = [(min_x - 2, min_y - 2), (min_x - 2, min_y + h + 2), (min_x + w + 2, min_y + h + 2), (min_x + w + 2, min_y - 2)]

   

    #     # print(min_x, max_x, min_y, max_y)
    #     # print(x, y, w, h)
        # draw.polygon(tuple(coordinates), outline = "white", width=1)

        # points = [x for x in markdown['controlPoints'] if x['label'] in ["CRV", "CAV", "CRD", "CAD"]]
        # coordinates_points = [(float(i['position'][0] / spacing[0]), float(i['position'][1]) / spacing[1]) for i in points]
        # point_size = 5
        # for point in coordinates_points:
        #     draw.ellipse([(point[0]-point_size, point[1]-point_size), (point[0]+point_size, point[1]+point_size)], fill="blue", width=1)

        # draw.polygon(tuple(coordinates2), outline = palette[markdown['name'][0]], width=2)
        # font = ImageFont.truetype('arial', size=22)
        # draw.text(
        #     (min_x + w + 30, min_y),  # Coordinates
        #     case + " " + markdown['name'],  # Text
        #     # markdown['name'],  # Text
        #     palette[markdown['name'][0]],  # Color
        #     font
        # )


        row = [filename, page_height, page_width, AuthorID, Overlapped, category, id, x, y, w, h]
        row1 = [filename, page_height, page_width, category, category1, x, y, w, h]

        # data_coordinates.append(row)
        data_coordinates.append(row1)

    # plt.subplot(1, 2, 2)     
    # plt.imshow(input)


            

# Create crops for U-Net Single Vertebra 
Also save original coordinates of right top corner

In [ ]:
data_coordinates
import pandas as pd
documents = pd.DataFrame(data_coordinates, columns=['filename', 'page_height', 'page_width', 'category', 'region', 'x', 'y', 'width', 'height'])
cases = documents['filename'].unique()



In [ ]:
path_to_source_dataset = os.path.join(path_to_dataset, "png")
path_to_source_dataset_label = os.path.join(path_to_dataset, "fill_no_FH")
path_to_source_dataset_dicom = os.path.join(path_to_dataset, "dicom")

path_to_destination_folder = os.path.join(path_to_dataset, "data_single_vertebra")


for case in cases:
    path_to_case_png = os.path.join(path_to_source_dataset, case)
    path_to_mask_png = os.path.join(path_to_source_dataset_label, case)

    markdowns = read_all_markdowns(os.path.join(path_to_dataset, *["labels", case.split(".")[0]]))

    image = cv2.imread(path_to_case_png, 1)
    dicom = pydicom.dcmread(os.path.join(path_to_source_dataset_dicom, case.split(".")[0] + ".dcm"))
    if spacing == None:
        spacing = dicom.ImagerPixelSpacing
    page_height, page_width = image.shape[:2]
    crops = documents[(documents.filename == case) & (documents.category == "Vertebrae")]

    for index, row in crops.iterrows():
        # print(row.filename.split(".")[0] + "_" + row.region)
        x, y = row.x, row.y
        w, h = row.width, row.height
        add = 15
        mask = Image.new('RGB', (page_width, page_height), 'black')
        draw = ImageDraw.Draw(mask)

        crop = image[y - add: y + h  + add, x - add: x + w + add]
        new_file_path = os.path.join(*[path_to_destination_folder, "images", row.filename.split(".")[0] + "_" + row.region + ".png"])
        cv2.imwrite(new_file_path, crop)

        vert = next(item for item in markdowns if item["name"] == row.region)
        coordinates = [(float(i['position'][0] / spacing[0]), float(i['position'][1]) / spacing[1]) for i in vert["controlPoints"]]
        draw.polygon(tuple(coordinates), fill='white', outline = 'white', width=1)
        crop_mask = np.asarray(mask)
        crop_mask = crop_mask[y - add: y + h  + add, x - add: x + w + add]
        new_file_path_mask = os.path.join(*[path_to_destination_folder, "labels", row.filename.split(".")[0] + "_" + row.region + ".png"])
        cv2.imwrite(new_file_path_mask, crop_mask) 

        


# Converting data to YOLOv5 format

In [ ]:
import pandas as pd
documents = pd.DataFrame(data_coordinates, columns=['filename', 'page_height', 'page_width', 'AuthorID', 'Overlapped', 'category', 'id', 'x', 'y', 'width', 'height'])
documents


# path_to_save = "C:\\Users\\gieko\\Dropbox\\NIITO_Vertebrae\\NIITO_Vertebrae_Dataset\\NIITO_Vertebrae_Dataset_Final_resized\\data_for_YOLO\\for_YOLO_Test.csv"
# documents[['filename', 'page_height', 'page_width', 'category', 'id', 'x', 'y', 'width', 'height', 'x_scaled', 'y_scaled', 'w_scaled', 'h_scaled']].to_csv(path_to_save, index=False, quotechar = ';')

**Scaling the image to reduce training time**  
To save on training time, resize the images to a maximum height and width of 640 and 273. While resizing the image, the bounding box cordinates also changes. This code computes how much each image is shrinken and updates the bounding box coordinates appropriately.

In [ ]:
import cv2
import os

# path_to_dataset_png = "C:\\Users\\gieko\\Dropbox\\NIITO_Vertebrae\\NIITO_Vertebrae_Dataset\\NIITO_Vertebrae_Dataset_Final_resized\\png"
# path_to_dataset_png_resized = "C:\\Users\\gieko\\Dropbox\\NIITO_Vertebrae\\NIITO_Vertebrae_Dataset\\NIITO_Vertebrae_Dataset_Final_resized\\png_h640_w273"

path_to_dataset_png = "C:\\Users\\gieko\\Dropbox\\NIITO_Vertebrae\\NIITO_Vertebrae_Dataset\\NIITO_Vertebrae_Dataset_Final_Test_resized\\png"
path_to_dataset_png_resized = "C:\\Users\\gieko\\Dropbox\\NIITO_Vertebrae\\NIITO_Vertebrae_Dataset\\NIITO_Vertebrae_Dataset_Final_Test_resized"

# path_to_documents_scaled = '/content/documents/documents/images'
# path_to_ids_scaled = '/content/documents/ids/images'
# path_to_laminated_scaled = '/content/documents/laminated/images'

def scale_image(df, path_to_source_folder, path_to_destination_folder):
    df_new = []
    filename = df.filename
    # page_height = df.page_height[0]
    X, Y, W, H = map(int, df.x), map(int, df.y), map(int, df.width), map(int, df.height)
    for file, x, y, w, h in zip(filename, X, Y, W, H):
        image_path = os.path.join(path_to_source_folder, file)
        img = cv2.imread(image_path, 1)
        page_height, page_width = img.shape[:2]
        # print(img.shape, page_height)
        img = img[: page_height - 107, :]
        page_height, page_width = img.shape[:2]
        # print(img.shape, page_height, page_width)

        max_height = 640
        max_width = 273

        # print(page_height, page_width)
        
        # computes the scaling factor
        if max_height < page_height or max_width < page_width:
            scaling_factor = max_height / float(page_height)
            # print("h: ", scaling_factor)
            if max_width/float(page_width) < scaling_factor:
                scaling_factor = max_width / float(page_width)
                # print("w: ", scaling_factor)
            # scale the image with the scaling factor
            img = cv2.resize(img, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)
            # print("@!", img.shape)
        jpg_filename = file.replace(file.split(".")[-1], 'jpg')
        new_file_path = os.path.join(path_to_destination_folder, jpg_filename)
        cv2.imwrite(new_file_path, img) # write the scales image
        
        # save new page height and width
        page_height, page_width = page_height*scaling_factor, page_width*scaling_factor
        # compute new x, y, w, h coordinates after scaling
        x, y, w, h= int(x*scaling_factor), int(y*scaling_factor), int(w*scaling_factor), int(h*scaling_factor)
        row = [jpg_filename, x, y, w, h, round(page_height), page_width]
        df_new.append(row)
    return df_new

scaled_data = scale_image(documents, path_to_dataset_png, path_to_dataset_png_resized)


In [ ]:
def add_data_abt_scaling_to_df(scaled_data, original_df):
  scaled = list(zip(*scaled_data))

  original_df['new_filename'] = scaled[0]
  original_df['x_scaled'] = scaled[1]
  original_df['y_scaled'] = scaled[2]
  original_df['w_scaled'] = scaled[3]
  original_df['h_scaled'] = scaled[4]
  original_df['page_height_scaled'] = scaled[5]
  original_df['page_width_scaled'] = scaled[6]
  return original_df


add_data_abt_scaling_to_df(scaled_data, documents)

path_to_save = "C:\\Users\\gieko\\Dropbox\\NIITO_Vertebrae\\NIITO_Vertebrae_Dataset\\NIITO_Vertebrae_Dataset_Final_resized\\data_for_YOLO\\for_YOLO_Test.csv"

documents[['filename', 'new_filename', 'page_height', 'page_width', 'page_height_scaled', 'page_width_scaled', 'AuthorID', 'Overlapped', 'category', 'id', 'x', 'y', 'width', 'height', 'x_scaled', 'y_scaled', 'w_scaled', 'h_scaled']].to_csv(path_to_save, index=False, quotechar = ';')


In [ ]:
classes_1 = {
    "C2": 0,
    "Vertebrae": 1,
    "FerumHead": 2
}

classes_2 = {
    "C": 0,
    "Th": 1,
    "L": 2,
    "S": 3,
    "FH": 4
}


def x_center(df):
  return int(df.x_scaled + (df.w_scaled/2))
def y_center(df):
  return int(df.y_scaled + (df.h_scaled/2))

def w_norm(df, col):
  return df[col]/df['page_width_scaled']
def h_norm(df, col):
  return df[col]/df['page_height_scaled']

def data_to_yolo(path_to_cleaned_csv):
  df = pd.read_csv(path_to_cleaned_csv, sep=';')
  
  df['labels'] = [classes_1[df['category'][i]] for i in range(len(df))]

  df['x_center'] = df.apply(x_center, axis=1)
  df['y_center'] = df.apply(y_center, axis=1)

  df['x_center_norm'] = df.apply(w_norm, col='x_center',axis=1)
  df['width_norm'] = df.apply(w_norm, col='w_scaled', axis=1)

  df['y_center_norm'] = df.apply(h_norm, col='y_center',axis=1)
  df['height_norm'] = df.apply(h_norm, col='h_scaled',axis=1)
  return df

documents_yolo = data_to_yolo(path_to_save)

**Segregate labels to destination folders**

In [ ]:
import shutil
import numpy as np

def segregate_labels(df, train_label_path):
  filenames = []
  for filename in df.filename:
    filenames.append(filename)
  filenames = set(filenames)
  for i, filename in enumerate(filenames):
    # print(i)
    name = filename.replace(filename.split('.')[-1], 'txt')
    yolo_list = []

    for _,row in df[df.filename == filename].iterrows():
      yolo_list.append([row.labels, row.x_center_norm, row.y_center_norm, row.width_norm, row.height_norm])

    yolo_list = np.array(yolo_list)
    # Save the .img & .txt files to the corresponding train and validation folders
    np.savetxt(os.path.join(train_label_path, name), yolo_list, fmt=["%d", "%f", "%f", "%f", "%f"])
path = "C:\\Users\\gieko\\Dropbox\\NIITO_Vertebrae\\NIITO_Vertebrae_Dataset\\NIITO_Vertebrae_Dataset_Final_resized\\data_for_YOLO\\labels"
segregate_labels(documents_yolo, path)

print(len(documents_yolo), len(os.listdir(path)))


1300 50
